In [ ]:
# November 8 2019
# development of function get_subtree
# Get an opentree synthetic tree for a rank X, from a clade Y,
# distributed in geographic region Z.

# arguments
# ott_ids: Numeric. If given only one, it will get a subtree for the corresponding node
         # If given 2 or more, it will give back an induced subtree
# rank: Character string. Indicates the taxonomic rank to search for.
# clade: Character string or number (?). Indicates a phylogenetic context to search for.
# range: Character string. Indicates the geogaphic region to search for.
# ott_version: Character string


In [ ]:
# define arguments
ott_ids = 494367
rank = genus
ott_version = "3.1"

In [1]:
# download the ott file
! wget 'http://files.opentreeoflife.org/ott/ott3.1/ott3.1.tgz'

--2019-11-08 17:05:13--  http://files.opentreeoflife.org/ott/ott3.1/ott3.1.tgz
Resolving files.opentreeoflife.org... 129.237.33.155
Connecting to files.opentreeoflife.org|129.237.33.155|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.opentreeoflife.org/ott/ott3.1/ott3.1.tgz [following]
--2019-11-08 17:05:13--  https://files.opentreeoflife.org/ott/ott3.1/ott3.1.tgz
Connecting to files.opentreeoflife.org|129.237.33.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106275258 (101M) [application/x-gzip]
Saving to: ‘ott3.1.tgz’

ott3.1.tgz          100%[===================>] 101.35M  20.5MB/s    in 5.4s    

2019-11-08 17:05:19 (18.7 MB/s) - ‘ott3.1.tgz’ saved [106275258/106275258]



In [2]:
# unzip it
import gzip
import shutil
with gzip.open('ott3.1.tgz', 'rb') as fin:
    with open('ott3.1.tsv', 'wb') as fout:
        shutil.copyfileobj(fin, fout)

In [3]:
# read the ott taxonomy file
# match 
# clean it
taxonomy_tsv = 'ott3.1red.tsv'

In [ ]:
fi = open(taxonomy_tsv).readlines()
ott_ids = []
for lin in fi:
    lii = lin.split("\t")
    ott_ids.append(lii[0])

In [ ]:
%load_ext rpy2.ipython